In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:

# Define the sliding window size and stride
# Define a PyTorch dataset to generate input/target pairs for the LSTM model
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size=7, stride=1):
        self.data = data
        self.window_size = window_size
        self.stride = stride

    def __len__(self):
        return len(self.data) - self.window_size

    def __getitem__(self, idx):
        inputs = self.data[idx:idx+self.window_size]
        target = self.data[idx+self.window_size]
        return inputs, target

# Define your LSTM model here with 6 LSTM layers and 1 fully connected layer
class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size,output_size, num_layers=6):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [3]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
def ExpSmooth(PeMS,alpha=0.2):
    
    # Apply exponential smoothing to the time serie
    for i in range(len(PeMS.columns)):
        y = PeMS[PeMS.columns[i]]
        model = ExponentialSmoothing(y).fit(smoothing_level=alpha)
        smooth = model.fittedvalues
        PeMS[PeMS.columns[i]] = smooth
    return PeMS

In [4]:
def my_data_loader(data, window_size = 7, stride = 1):
    dataset = TimeSeriesDataset(data.values, window_size, stride)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    loader = [(inputs.to(device), targets.to(device)) for inputs, targets in loader]
    return loader
    

In [6]:
def experiment_dataset(cluster_size, df, train_len=0):
    cluster_dict={"size":cluster_size}
    for i in range(len(PeMS.columns)+1-cluster_size):
        model = LSTMModel(input_size=cluster_size, hidden_size=32, num_layers=layers, output_size=cluster_size)
        if train_len == 0 :
            train_data= df[df.columns[i:i+cluster_size]][:'2018-02-10 00:00:00']
            val_data =  df[df.columns[i:i+cluster_size]]['2018-02-10 00:00:00':'2018-02-14 00:00:00']
            test_data = df[df.columns[i:i+cluster_size]]['2018-02-14 00:00:00':] 
        else :
            train_data= df[df.columns[i:i+cluster_size]][:train_len]
            val_data =  df[df.columns[i:i+cluster_size]][train_len: train_len*1.2]
            test_data = df[df.columns[i:i+cluster_size]][train_len*1.2:train_len*1.4] 

        train_loader = my_data_loader(train_data)
        val_loader = my_data_loader(val_data)
        test_loader = my_data_loader(test_data) 
            
        cluster_dict[i]={"model":model,"train":train_loader,"val":val_loader,"test":test_loader}
    with open('./experiment/clusterS{}l{}.pkl'.format(cluster_size,train_len), 'wb') as f:
        pickle.dump(cluster_dict, f)